from Container import *
from utils import *
from Recommender import *
from pipeline import *
from datetime import datetime, timedelta

In [2]:
from tqdm.notebook import tqdm

In [16]:
import warnings
warnings.filterwarnings('ignore')

In [14]:
def run_on_history(model, events):
    from_date = events.date.min()
    end_date = events.date.max()

    to_date = timedelta(440, 0) + from_date
    init = events[(events.date < to_date) & (events.date > from_date)]

    model.initialize(init)

    preds = []
    targets = []
    
    
    for i in tqdm(range(10000)):
#     while from_date <= end_date:
        
        from_date = to_date
        to_date = from_date + timedelta(10, 0)
        
        if from_date > end_date:
            break
        cur_events = events[(events.date < to_date) & (events.date > from_date)]

        pred, target = model.process(cur_events)
        preds += pred
        targets += target

    return preds, targets


In [9]:
d = 'beam'

In [10]:
data, files, users = get_data(d)

In [12]:
model = SparseRecommender(files, users)

In [17]:
# ~ 25 минут на все
results = run_on_history(model, data)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
res2 = np.zeros(5)
cnt = 0

In [ ]:
for i in range(len(targets)):
    if len(targets[i]) > 0:
        res2 += count_metrics(preds[i], targets[i])
        cnt += 1

In [25]:
def calculate_metrics_on_review(pred, target):
    res = np.zeros(5)
    for i, k in enumerate([1, 3, 5, 10]):
        if len(set(pred[:k]).intersection(set(target))):
            res[i] += 1
    rr = np.inf
    for t in target:
        rr = min(rr, 1 + np.where(pred == t)[0])
    res[4] = 1 / rr
    return res

In [26]:
def calculate_metrics(results):
    res = np.zeros(5)
    cnt = 0
    for pred, target in zip(*results):
        if len(target) > 0:
            res += calculate_metrics_on_review(pred, target)
            cnt += 1
    return res / cnt

In [37]:
metrics = calculate_metrics((preds, results[1]))

In [38]:
metrics

array([0.00617284, 0.02555268, 0.05656044, 0.2474878 , 0.04451497])

In [4]:
dfs = get_df('beam')

In [6]:
dfs.keys()

dict_keys(['reviewer', 'commit_file', 'pull', 'participant', 'pull_author', 'pull_file', 'commit_author', 'file', 'commit', 'user'])

In [16]:
dfs['user']

,login,name_github,email_github,name_git,email_git
0,0xmax,['Max Buschner'],[],[],[]
1,1028332163,['lasynchronous'],['1028332163@qq.com'],['1028332163'],['1028332163@qq.com']
2,11moon11,['Kirill Kozlov'],[],"['kirillkozlov', 'Kirill Kozlov']","['kirillkozlov@google.com', '8753059+11moon11@..."
3,397090770,['wyp'],[],['iteblog'],['wyphao.2007@163.com']
4,=:=,[],[],['='],['=']
...,...,...,...,...,...
1005,zongweiz,[],[],['Zongwei Zhou'],['zongweiz@google.com']
1006,zorro786,['Abdul Qadeer'],[],[],[]
1007,zouabimourad,['Mourad ZOUABI'],[],[],[]
1008,zoyahav,['Zohar Yahav'],[],[],[]
